In [1]:
pip install langchain-community

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="meta-llama/llama-4-scout-17b-16e-instruct"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The best answer is  Neil Armstrong


In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/principal-software-engineering/job/R-63381")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























Principal, Software Engineering










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Me

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [9]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Principal, Software Engineering',
 'experience': ['10+ years experience as a software engineer, data engineer, or architect building, designing and coding in a distributed data management systems for a large-scale data architecture',
  '5+ years experience working with Hadoop and Big Data processing frameworks (Spark, Hive, NiFi, Spark-Streaming, Flink, etc.)'],
 'skills': ['Spark',
  'SQL',
  'NoSQL',
  'cloud platforms (e.g., AWS, Azure, Google Cloud)',
  'Python',
  'Scala',
  'Java',
  'GitHub',
  'CI/CD',
  'DevOps',
  'Airflow',
  'RESTful APIs'],
 'description': 'We are looking for a talented Principal Software Engineer to join our Global Technology Data & Artificial Intelligence (D&AI) organization to guide and influence multiple engineering teams to:n'}

In [10]:
type(json_res)

dict

In [15]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [11]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [12]:
job = json_res
job['skills']

['Spark',
 'SQL',
 'NoSQL',
 'cloud platforms (e.g., AWS, Azure, Google Cloud)',
 'Python',
 'Scala',
 'Java',
 'GitHub',
 'CI/CD',
 'DevOps',
 'Airflow',
 'RESTful APIs']

In [13]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfoli

In [14]:
job

{'role': 'Principal, Software Engineering',
 'experience': ['10+ years experience as a software engineer, data engineer, or architect building, designing and coding in a distributed data management systems for a large-scale data architecture',
  '5+ years experience working with Hadoop and Big Data processing frameworks (Spark, Hive, NiFi, Spark-Streaming, Flink, etc.)'],
 'skills': ['Spark',
  'SQL',
  'NoSQL',
  'cloud platforms (e.g., AWS, Azure, Google Cloud)',
  'Python',
  'Scala',
  'Java',
  'GitHub',
  'CI/CD',
  'DevOps',
  'Airflow',
  'RESTful APIs'],
 'description': 'We are looking for a talented Principal Software Engineer to join our Global Technology Data & Artificial Intelligence (D&AI) organization to guide and influence multiple engineering teams to:n'}

In [16]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Sushil Soni, a business development executive at Learning Links Foundation. Learning Links Foundation is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of LLF 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase LLF's portfolio: {link_list}
        Remember you are Sushil Soni, BDE at Learning Links Foundation. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Guidance for Your Principal Software Engineer Role

Dear Hiring Manager,

I came across the job description for a Principal software engineer at your organization and was impressed by the scale and complexity of the data architecture you're looking to build and manage. As a Business Development Executive at Learning Links Foundation (LLF), I'd like to highlight our capabilities in fulfilling your needs.

With over a decade of experience in AI and software consulting, we've empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise in large-scale data architecture, big data processing frameworks, and cloud platforms makes us an ideal partner for your project.

Our team has extensive experience with Hadoop and Big Data processing frameworks, including Spark, Hive, NiFi, Spark-Streaming, and Flink. We've successfully delivered projects involving Spark, SQL, NoSQL, clo